In [8]:
import json
from github import Github, Auth, GithubException
from datetime import datetime, timedelta, timezone


def update_github_repositories(access_token, json_filename):
  # 1. Setup Authentication
  auth = Auth.Token(access_token)
  g = Github(auth=auth)

  # 2. Rate Limit Reporting
  rate_limit = g.get_rate_limit()
  reset_time_utc = rate_limit.rate.reset
  limit = rate_limit.rate.limit
  remaining = rate_limit.rate.remaining

  # Manual IST Conversion (UTC + 5:30)
  ist_offset = timezone(timedelta(hours=5, minutes=30))
  ist_reset = reset_time_utc.astimezone(ist_offset)

  print("="*40)
  print("GITHUB API RATE LIMIT STATUS")
  print(f"Cap (Total Limit): {limit}")
  print(f"Requests Remaining: {remaining}")
  print(f"Reset Time (IST): {ist_reset.strftime('%Y-%m-%d %I:%M:%S %p')}")
  print("="*40)
  print("Starting process...\n")

  # 3. Process the File
  with open(json_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

  user = g.get_user()

  for category, repos in data.items():
    print(f"--- Category: {category} ---")
    for entry in repos:
      old_name = entry.get('current_name')
      new_name = entry.get('new_name')
      new_desc = entry.get('new_description')
      new_topics = entry.get('topics', [])

      try:
        # Fetch the repo using current name
        repo = user.get_repo(old_name)

        # Update Name and Description
        repo.edit(name=new_name, description=new_desc)

        # Update Topics
        repo.replace_topics(new_topics)

        print(f"✔ Successfully updated: {old_name} -> {new_name}")

      except GithubException as e:
        print(f"✘ GitHub Error on {old_name}: {e.data.get('message', 'Unknown Error')}")
      except Exception as e:
        print(f"✘ Unexpected error on {old_name}: {e}")

  print("\nAll tasks completed.")

https://github.com/settings/tokens

In [9]:
update_github_repositories(
    access_token="PERSONAL_ACCESS_TOKENS_CLASSIC",
    json_filename="./updates.json"
)

BadCredentialsException: 401 {"message": "Bad credentials", "documentation_url": "https://docs.github.com/rest", "status": "401"}